# Import

In [1]:
import gdown

# Specify the file ID and output path
file_id = "1-0A3obqFnABamVwIRGBKTgG2mQ1mrtI2"
destination = "/content/train_data.csv"  # Change the filename and extension if necessary

# Download the file
gdown.download(f"https://drive.google.com/uc?id={file_id}", destination, quiet=False)

print("File downloaded to:", destination)

Downloading...
From: https://drive.google.com/uc?id=1-0A3obqFnABamVwIRGBKTgG2mQ1mrtI2
To: /content/train_data.csv
100%|██████████| 7.75M/7.75M [00:00<00:00, 47.3MB/s]

File downloaded to: /content/train_data.csv


In [2]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import accuracy_score

In [6]:
# # # Define preprocessing functions
# import nltk
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [7]:
from HelperFunctions import *
from Preprocessing import preprocess

In [8]:
train_data = pd.read_csv('/content/train_data.csv')
# test_data = pd.read_csv('Data/test.csv')
test_data = None

In [9]:
train_data.head(2)

,SampleID,Discussion,Category
0,1,"Without sitting down and doing it manually, yo...",Sports
1,2,All your Search ends with this link.,STEM


In [ ]:
# test_data.head(3)

# Preprocessing

In [11]:
# !pip install nltk
# import nltk
# nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [12]:
pre_method = 2
X_train_padded, Y_train, X_test_padded, num_unique_words, max_sequence_length = preprocess(train_data=train_data,
                                                                                            test_data=test_data,
                                                                                            pre_method=pre_method,
                                                                                            fx_opt=2)

Drop Nan...
	train_data.shape before (24989, 3)
	train_data.shape after (24646, 3)
start preprocessing...
Encoding Y_train...
Calc unique words...
	Num of Unique words: 59373
Tokenizer...


In [13]:
print(f"X_train.shape: {X_train_padded.shape}")
# print(f"X_test.shape: {X_test_padded.shape}")
print(f"Y_train.shape: {Y_train.shape}")

X_train.shape: (24646, 809)
Y_train.shape: (24646,)


In [14]:
Y_train.head(3)

,Category
0,1
1,4
2,4


# RNN

In [15]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout

In [16]:
print(f"num_unique_words: {num_unique_words}")
print(f"max_sequence_length: {max_sequence_length}")

num_unique_words: 59373
max_sequence_length: 809


In [22]:
num_classes = 5
# Define the model
rnn_model = Sequential()

# Embedding layer to convert word indices into dense vectors of fixed size
rnn_model.add(Embedding(input_dim=num_unique_words, output_dim=128, input_length=max_sequence_length))

# RNN layer (SimpleRNN)
rnn_model.add(SimpleRNN(units=64, return_sequences=False))

# Dropout for regularization
rnn_model.add(Dropout(0.5))

# Fully connected layer for classification
rnn_model.add(Dense(5, activation='softmax'))  # Adjust output units for the number of classes

# Explicitly build the model
rnn_model.build(input_shape=(None, max_sequence_length))

# Compile the model
rnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
rnn_model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)              │ (None, 809, 128)            │       7,599,744 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_4 (SimpleRNN)             │ (None, 64)                  │          12,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 5)                   │             325 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,612,421 (29.04 MB)

 Trainable params: 7,612,421 (29.04 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
# Assuming Y_train contains labels in integer form (e.g., [0, 1, 2, 3, 4])
Y_train_categorical = to_categorical(Y_train, num_classes=num_classes)

In [23]:
epochs = 5
batch_size = 32
validation_split = 0.2

In [24]:
rnn_model.fit(X_train_padded, Y_train_categorical, epochs=epochs, batch_size=batch_size, validation_split=validation_split)

Epoch 1/5
617/617 ━━━━━━━━━━━━━━━━━━━━ 66s 102ms/step - accuracy: 0.2083 - loss: 1.6639 - val_accuracy: 0.2245 - val_loss: 1.6100
Epoch 2/5
617/617 ━━━━━━━━━━━━━━━━━━━━ 71s 87ms/step - accuracy: 0.2048 - loss: 1.6568 - val_accuracy: 0.2195 - val_loss: 1.6038
Epoch 3/5
617/617 ━━━━━━━━━━━━━━━━━━━━ 82s 87ms/step - accuracy: 0.2216 - loss: 1.6147 - val_accuracy: 0.2201 - val_loss: 1.6033
Epoch 4/5
617/617 ━━━━━━━━━━━━━━━━━━━━ 82s 87ms/step - accuracy: 0.2265 - loss: 1.6085 - val_accuracy: 0.2223 - val_loss: 1.6052
Epoch 5/5
617/617 ━━━━━━━━━━━━━━━━━━━━ 81s 86ms/step - accuracy: 0.2194 - loss: 1.6079 - val_accuracy: 0.2197 - val_loss: 1.6032


In [ ]:
# Predict the labels for the test set
predictions = rnn_model.predict(X_train_padded)

# If it's a multi-class classification task, get the predicted class for each sample
predicted_classes = np.argmax(predictions, axis=1)

## Evaluation

### Train

In [ ]:
Y_train_pred = ffnn_model.predict(X = X_train_padded)

In [ ]:
train_accuracy = accuracy_score(Y_train_pred, Y_train)
print(f"Train Accuracy: {train_accuracy}")

In [ ]:
file_name = f'RNN-m{pre_method}-e{epochs}-a{int(train_accuracy*100)}'
rnn_model.save(f'Models/RNN/{file_name}.h5')

### Test

In [ ]:
# Y_test_pred = ffnn_model.predict(X = X_test)

In [ ]:
# file_name = f'FFNN-m{pre_method}-e{epochs}'
# save_csv(data=Y_test_pred, file_name=file_name, header=['SampleID', 'Category'], numbering=True)